In [1]:
from datasets import concatenate_datasets, load_dataset, load_from_disk

# bookcorpus = load_dataset("bookcorpus", split="train")
bookcorpus = load_from_disk("../processed_datadir/bookcorpus-raw-train")



In [2]:
for i in range(10):
    print(bookcorpus[i]["text"])
    print("--------")

usually , he would be tearing around the living room , playing with his toys .
--------
but just one look at a minion sent him practically catatonic .
--------
that had been megan 's plan when she got him dressed earlier .
--------
he 'd seen the movie almost by mistake , considering he was a little young for the pg cartoon , but with older cousins , along with her brothers , mason was often exposed to things that were older .
--------
she liked to think being surrounded by adults and older kids was one reason why he was a such a good talker for his age .
--------
`` are n't you being a good boy ? ''
--------
she said .
--------
mason barely acknowledged her .
--------
instead , his baby blues remained focused on the television .
--------
since the movie was almost over , megan knew she better slip into the bedroom and finish getting ready .
--------


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../tokenizer_save/tokenizer-gpt2-2048/")

In [4]:


raw_datasets = bookcorpus

In [5]:
from transformers import AutoTokenizer
import multiprocessing

# load tokenizer

num_proc = multiprocessing.cpu_count()-4
print("num_proc",num_proc)
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    tokenized_inputs = tokenizer(
       examples["text"], 
       #return_special_tokens_mask=True, 
       #truncation=True, max_length=tokenizer.model_max_length  # 这里不应该有这句话
    )
    return tokenized_inputs

# preprocess dataset
tokenized_datasets = raw_datasets.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features


num_proc 12
The max length for the tokenizer is: 2048


Map (num_proc=12):   0%|          | 0/74004228 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2363 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5245 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3081 > 2048). Running this sequence through the model will result in indexing errors


{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [6]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= tokenizer.model_max_length:
        total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=num_proc)
# shuffle dataset
tokenized_datasets = tokenized_datasets.shuffle(seed=34)

print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")
# the dataset contains in total 3417216000 tokens


Map (num_proc=12):   0%|          | 0/74004228 [00:00<?, ? examples/s]

the dataset contains in total 1008785408 tokens


In [7]:
from datasets import load_dataset, DatasetDict, load_from_disk
splits = tokenized_datasets.train_test_split(test_size=0.03)
preprocessed_splits = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],

})

In [8]:
preprocessed_splits.save_to_disk("../processed_datadir2/bookcorpus-gpt2-2048/")

Flattening the indices:   0%|          | 0/477793 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/477793 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/14778 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14778 [00:00<?, ? examples/s]